In [58]:
import pandas as pd

In [59]:
cleaned_league = pd.read_csv("complete_cleaned.csv")
counter_df = pd.read_csv("counters.csv")

In [60]:
cleaned_league = cleaned_league.drop(columns=['Unnamed: 0.1', "Unnamed: 0"])
cleaned_league.iloc[:2]

,gameid,side,pick1,pick2,pick3,pick4,pick5,ban1,ban2,ban3,ban4,ban5,result
0,FRA1TMNT1/450135,Blue,Rumble,Jarvan IV,Syndra,Sivir,Annie,Alistar,Azir,Irelia,NaN,NaN,0
1,FRA1TMNT1/450135,Red,Gnar,Rengar,Ahri,Graves,Janna,Rek'Sai,LeBlanc,Kassadin,NaN,NaN,1


In [61]:
counter_df.head()

,Champion,counters champ,champ counters
0,nilah,"('Sivir', 47)","('Aphelios', 56)"
1,nilah,"('Smolder', 48)","('Draven', 56)"
2,nilah,"('Xayah', 50)","('Samira', 58)"
3,nilah,"('Zeri', 50)","(""Kai'Sa"", 58)"
4,nilah,"('Jinx', 51)","('Twitch', 58)"


In [65]:
def get_picks(df): #input df will be two rows, one for blue and one for red per gameid 

    blue = df[df['side'] == 'Blue'][['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].values.flatten().tolist()
    red = df[df['side'] == 'Red'][['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].values.flatten().tolist()

    blue = [champ.lower().replace(" ", "-").replace("'", "").replace("nunu-&-willump", "nunu-willump").replace("dr.-mundo", "dr-mundo") for champ in blue]  # Convert all blue picks to lowercase
    red = [champ.lower().replace(" ", "-").replace("'", "").replace(" ", "-").replace("'", "") for champ in red]    # Convert all red picks to lowercase

    num_counters = count_counters([red, blue])
    counter_data = {"Red": num_counters[0], "Blue": num_counters[1]}
    df["Number of Counter Picks"] = df["side"].map(counter_data)
    return df


#note that multiple characters on one team can counter one character on another, but we will count as 1 instead of 2. 
#ex) if red team has rengar, and blue team has a twitch and evelyn. If twich and evelyn BOTH counter rengar, we would set blue teams counter to be +1
    #although it would be a stronger counter than if blue had only one counter, it is usually the case that similar lanes counter each other,
    #thus it is strong to assume that for most games, each counter has a max of 5 and min of 0

def count_counters(team): 
    counter_df = pd.read_csv("counters.csv")
    red = team[0]
    blue = team[1]

    countered_red_champs = set()
    countered_blue_champs = set()

    # Count blue team's counter picks (against red team)
    for red_champ in red:
        champ_counters = set(counter_df[counter_df['Champion'] == red_champ]["counters champ"].dropna().tolist())
        for blue_champ in blue:
            if blue_champ in champ_counters:
                countered_red_champs.add(red_champ)  # Mark red champ as countered
                break  # Only count one counter per champion

    num_blue_counters = len(countered_red_champs)

    # Count red team's counter picks (against blue team)
    for blue_champ in blue:
        champ_counters = set(counter_df[counter_df['Champion'] == blue_champ]["counters champ"].dropna().tolist())
        for red_champ in red:
            if red_champ in champ_counters:
                countered_blue_champs.add(blue_champ)  # Mark blue champ as countered
                break  # Only count one counter per champion

    num_red_counters = len(countered_blue_champs)

    return [num_red_counters, num_blue_counters]  

pract = cleaned_league[:2]
pract.groupby(["gameid", "side"]).apply(get_picks)

C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_20800\488099222.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pract.groupby(["gameid", "side"]).apply(get_picks)


gameid  side   pick1      pick2   pick3  \
gameid           side                                                        
FRA1TMNT1/450135 Blue 0  FRA1TMNT1/450135  Blue  Rumble  Jarvan IV  Syndra   
                 Red  1  FRA1TMNT1/450135   Red    Gnar     Rengar    Ahri   

                          pick4  pick5     ban1     ban2      ban3 ban4 ban5  \
gameid           side                                                          
FRA1TMNT1/450135 Blue 0   Sivir  Annie  Alistar     Azir    Irelia  NaN  NaN   
                 Red  1  Graves  Janna  Rek'Sai  LeBlanc  Kassadin  NaN  NaN   

                         result  Number of Counter Picks  
gameid           side                                     
FRA1TMNT1/450135 Blue 0       0                        0  
                 Red  1       1                        0

In [68]:
counter_df[counter_df['Champion'] == 'gwen']

,Champion,counters champ,champ counters
475,gwen,"('Sivir', 47)","('Aphelios', 56)"
476,gwen,"('Smolder', 48)","('Draven', 56)"
477,gwen,"('Xayah', 50)","('Samira', 58)"
478,gwen,"('Zeri', 50)","(""Kai'Sa"", 58)"
479,gwen,"('Jinx', 51)","('Twitch', 58)"
